In [2]:
import warnings
warnings.filterwarnings('ignore')

In [3]:
from tensorflow import keras
from keras.layers import Input, Lambda, Dense, Flatten
from keras.models import Model
from keras.applications.vgg16 import VGG16
from keras.applications.vgg16 import preprocess_input
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
import numpy as np
from glob import glob
import matplotlib.pyplot as plt

In [4]:
IMAGE_SIZE = [224, 224]

train_path = '/kaggle/input/chest-xray-pneumonia/chest_xray/train'
valid_path = '/kaggle/input/chest-xray-pneumonia/chest_xray/test'

In [5]:
vgg = VGG16(input_shape=IMAGE_SIZE + [3], weights='imagenet', include_top=False)

58889256/58889256 [==============================] - 0s 0us/step


In [6]:
for layer in vgg.layers:
    layer.trainable = False

In [7]:
folders = glob('/kaggle/input/chest-xray-pneumonia/chest_xray/train/*')
x = Flatten()(vgg.output)

In [8]:
prediction = Dense(len(folders), activation='softmax')(x)
# create a model object
model = Model(inputs=vgg.input, outputs=prediction)
# view the structure of the model
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0     

In [9]:
model.compile(
  loss='categorical_crossentropy',
  optimizer='adam',
  metrics=['accuracy']
)

In [10]:
from keras.preprocessing.image import ImageDataGenerator

In [11]:
train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)

test_datagen = ImageDataGenerator(rescale = 1./255)




# Make sure you provide the same target size as initialied for the image size
training_set = train_datagen.flow_from_directory('/kaggle/input/chest-xray-pneumonia/chest_xray/train',
                                                 target_size = (224, 224),
                                                 batch_size = 10,
                                                 class_mode = 'categorical')




test_set = test_datagen.flow_from_directory('/kaggle/input/chest-xray-pneumonia/chest_xray/test',
                                            target_size = (224, 224),
                                            batch_size = 10,
                                            class_mode = 'categorical')

Found 5216 images belonging to 2 classes.
Found 624 images belonging to 2 classes.


In [13]:
r = model.fit_generator(
  training_set,
  validation_data=test_set,
  epochs=5,
  steps_per_epoch=len(training_set),
  validation_steps=len(test_set)
)

Epoch 1/5
522/522 [==============================] - 117s 225ms/step - loss: 350.2091 - accuracy: 0.8248 - val_loss: 257.4603 - val_accuracy: 0.7869
Epoch 2/5
522/522 [==============================] - 116s 222ms/step - loss: 238.3386 - accuracy: 0.8683 - val_loss: 263.5730 - val_accuracy: 0.8526
Epoch 3/5
522/522 [==============================] - 116s 222ms/step - loss: 224.3756 - accuracy: 0.8836 - val_loss: 291.5619 - val_accuracy: 0.8542
Epoch 4/5
522/522 [==============================] - 114s 219ms/step - loss: 319.7373 - accuracy: 0.8627 - val_loss: 397.8585 - val_accuracy: 0.8365
Epoch 5/5
522/522 [==============================] - 114s 218ms/step - loss: 273.0132 - accuracy: 0.8796 - val_loss: 1049.5616 - val_accuracy: 0.6795


In [14]:
import tensorflow as tf
from keras.models import load_model

model.save('pneumonia_model.h5')

In [15]:
from keras.models import load_model

In [21]:
from tensorflow.keras.preprocessing import image

In [17]:
from keras.applications.vgg16 import preprocess_input

In [18]:
import numpy as np

In [19]:
model=load_model('/kaggle/working/pneumonia_model.h5')

In [25]:
image_path = '/kaggle/input/chest-xray-pneumonia/chest_xray/val/NORMAL/NORMAL2-IM-1440-0001.jpeg'
img = image.load_img(image_path, target_size=(224, 224))
x=image.img_to_array(img)
x=np.expand_dims(x, axis=0)
img_data=preprocess_input(x)
classes=model.predict(img_data)
result=int(classes[0][0])
print(classes)
print(result)

if result==1:
    print("Person is Normal")
else:
    print("Person is Affected By PNEUMONIA")

1/1 [==============================] - 0s 20ms/step
[[1. 0.]]
1
Person is Normal


In [26]:
image_path = '/kaggle/input/chest-xray-pneumonia/chest_xray/val/PNEUMONIA/person1949_bacteria_4880.jpeg'
img = image.load_img(image_path, target_size=(224, 224))
x=image.img_to_array(img)
x=np.expand_dims(x, axis=0)
img_data=preprocess_input(x)
classes=model.predict(img_data)
result=int(classes[0][0])
print(classes)
print(result)

if result==1:
    print("Person is Normal")
else:
    print("Result is Affected By PNEUMONIA")

1/1 [==============================] - 0s 19ms/step
[[1. 0.]]
1
Person is Normal
